In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import datetime
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn import linear_model
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import cross_val_score
import time

%load_ext autoreload
%autoreload 2
from functions import *
from plots import *

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8  # plotsize 

# Import traffic data

In [38]:
df_traf = pd.DataFrame.from_csv('../data_ski_in_solitude/berthoud_skiers_14_15.csv', parse_dates=True, index_col=0)

In [39]:
df_traf.head()

,away_WP,to_WP,day_of_week,skiers
FormattedDate,,,,
2014-11-14,0,1632,4,1632
2014-11-15,1695,1964,5,1901
2014-11-16,3390,1489,6,0
2014-11-17,1716,1493,0,-223
2014-11-18,1544,1531,1,-236


# Import snow data

In [40]:
df_met = pd.DataFrame.from_csv('../data_ski_in_solitude/clean_wp_14_15.csv', parse_dates=True, index_col=0)

In [41]:
resamp_time = '1440Min'
df_met = df_met.resample(resamp_time)

In [42]:
df_met.head()

,new_24,basedepth,open_trails,open_lifts,open_percent,tot_snow,week_snow_tot,2day_snow_tot,3day_snow_tot,4day_snow_tot,5day_snow_tot,6day_snow_tot
valid_at,,,,,,,,,,,,
2014-11-15,5,18,3,3,3,0,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-16,10,18,5,4,4,10,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-17,0,18,6,4,5,10,NaN,15,NaN,NaN,NaN,NaN
2014-11-18,0,18,7,4,1,10,NaN,10,15,NaN,NaN,NaN
2014-11-19,0,18,8,4,6,10,NaN,0,10,15,NaN,NaN


## Combine the traffic and met data.

In [47]:
df_all = df_traf.join(df_met, how='outer')

In [48]:
df_all.head()

,away_WP,to_WP,day_of_week,skiers,new_24,basedepth,open_trails,open_lifts,open_percent,tot_snow,week_snow_tot,2day_snow_tot,3day_snow_tot,4day_snow_tot,5day_snow_tot,6day_snow_tot
2014-11-14,0,1632,4,1632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-15,1695,1964,5,1901,5,18,3,3,3,0,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-16,3390,1489,6,0,10,18,5,4,4,10,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-17,1716,1493,0,-223,0,18,6,4,5,10,NaN,15,NaN,NaN,NaN,NaN
2014-11-18,1544,1531,1,-236,0,18,7,4,1,10,NaN,10,15,NaN,NaN,NaN


In [49]:
df_all = df_all[1:]

In [50]:
df_all

,away_WP,to_WP,day_of_week,skiers,new_24,basedepth,open_trails,open_lifts,open_percent,tot_snow,week_snow_tot,2day_snow_tot,3day_snow_tot,4day_snow_tot,5day_snow_tot,6day_snow_tot
2014-11-15,1695,1964,5,1901,5.000000,18,3,3,3.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-16,3390,1489,6,0,10.000000,18,5,4,4.0,10.000000,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-17,1716,1493,0,-223,0.000000,18,6,4,5.0,10.000000,NaN,15.000000,NaN,NaN,NaN,NaN
2014-11-18,1544,1531,1,-236,0.000000,18,7,4,1.0,10.000000,NaN,10.000000,15.000000,NaN,NaN,NaN
2014-11-19,1702,1714,2,-224,0.000000,18,8,4,6.0,10.000000,NaN,0.000000,10.000000,15.000000,NaN,NaN
2014-11-20,1692,1861,3,-55,0.000000,18,11,4,8.0,10.000000,NaN,0.000000,0.000000,10.000000,15.000000,NaN
2014-11-21,1965,3287,4,1267,0.000000,18,11,4,8.0,10.000000,NaN,0.000000,0.000000,0.000000,10.000000,15.000000
2014-11-22,2360,3125,5,2032,0.000000,18,12,4,9.0,10.000000,15.000000,0.000000,0.000000,0.000000,0.000000,10.000000
2014-11-23,2857,1940,6,1115,6.000000,18,12,4,9.0,16.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014-11-24,1952,1965,0,1128,4.000000,22,12,4,9.0,20.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000


In [51]:
df_all.tail()

,away_WP,to_WP,day_of_week,skiers,new_24,basedepth,open_trails,open_lifts,open_percent,tot_snow,week_snow_tot,2day_snow_tot,3day_snow_tot,4day_snow_tot,5day_snow_tot,6day_snow_tot
2015-04-22,1958,1950,2,-4053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-04-23,1747,1977,3,-3823,0,76,99,9,69,245,23,4,7,12,23,23
2015-04-24,2223,3877,4,-2169,0,75,97,9,68,245,23,0,4,7,12,23
2015-04-25,2606,3399,5,-1376,0,72,97,10,68,245,23,0,0,4,7,12
2015-04-26,4610,2103,6,-3883,5,74,97,10,68,250,12,0,0,0,4,7


# Export Combined Dataframe

In [52]:
df_all.to_csv('../data_ski_in_solitude/combined_14_15.csv')